
# Information about the system & environment

In [16]:
from datetime import datetime
import os
import platform
import psutil
import sh
import sys

Note that `psutil` is not in Python's standard library.

## Operating system

Using the `platform` module, it is easy to obtain information on the platform the Python interpreter is running on.

Information about the machine name:

In [9]:
platform.node()

'ubuntu'

The architecture and hardware:

In [16]:
platform.processor()

'x86_64'

In [13]:
sys.byteorder

'little'

In [19]:
os.cpu_count()

1

In [20]:
os.sched_getaffinity(0)

{0}

The operating system:

In [10]:
platform.system()

'Linux'

In [14]:
platform.release()

'4.15.0-38-generic'

In [15]:
platform.version()

'#41-Ubuntu SMP Wed Oct 10 10:59:38 UTC 2018'

In [17]:
platform.linux_distribution()

('debian', 'buster/sid', '')

In [6]:
platform.platform()

'Linux-4.15.0-38-generic-x86_64-with-debian-buster-sid'

## Numerics

The properties of floating point numbers can be obtained easily from the `sys.floatinfo` object.

The largest floating point value that can be represented, the smallest positive non-zero value:

In [4]:
print(sys.float_info.max, sys.float_info.min)

1.7976931348623157e+308 2.2250738585072014e-308


The number of significant digits of a floating point value:

In [2]:
sys.float_info.dig

15

## Processes

Detailed information is available on the processes running on the system.

In [8]:
for process in psutil.process_iter():
    if 'bash' in process.name():
        cpu_times = process.cpu_times()
        thread_str = f'threads: {process.num_threads()}'
        cpu_str = f'user: {cpu_times.user}, sys: {cpu_times.system}'
        print(f'{process.pid}: {process.name()} ({thread_str}, {cpu_str})')

10966: bash (threads: 1, user: 0.09, sys: 0.03)
13085: bash (threads: 1, user: 0.05, sys: 0.03)


CPU times are cumulative over the process' life time.

In [28]:
for process in psutil.process_iter():
    if process.cpu_times().user > 2.5:
        print(f'{process.name()}: {process.cpu_times().user}')

vmtoolsd: 18.13
snapd: 4.69
dbus-daemon: 3.19
dockerd: 45.21
docker-containerd: 26.17
gnome-shell: 30.3
Xwayland: 20.83
packagekitd: 23.36
gsd-color: 6.5
Xorg: 60.43
gnome-shell: 1731.97
ibus-daemon: 22.12
gsd-color: 6.18
vmtoolsd: 43.48
nautilus-desktop: 5.89
ibus-engine-simple: 7.62
gnome-software: 9.81
gnome-terminal-server: 7.03
ZMQbg/1: 10.69
chrome: 138.51
chrome: 44.74
chrome: 532.83
chrome: 3.39
chrome: 26.69
zeal: 5.41
chrome: 13.82


It is easy to kill processes, so you might want to be careful.

In [13]:
sleep = sh.sleep(120, _bg=True)

In [14]:
for process in psutil.process_iter():
    if 'sleep' in process.name():
        print(process)

psutil.Process(pid=13444, name='sleep', started='07:04:55')
psutil.Process(pid=13467, name='sleep', started='07:05:46')


In [15]:
name = 'sleep'
killed_pids = []
for process in psutil.process_iter():
    if name == process.name():
        print(f'killing {name}...')
        killed_pids.append(process.pid)
        process.kill()
print('killed: ', ', '.join(map(str, killed_pids)))

killing sleep...
killed:  13467


Exception in thread background thread for pid 13467:
Traceback (most recent call last):
  File "/home/gjb/miniconda3/envs/py36/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/gjb/miniconda3/envs/py36/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/gjb/miniconda3/envs/py36/lib/python3.6/site-packages/sh.py", line 1540, in wrap
    fn(*args, **kwargs)
  File "/home/gjb/miniconda3/envs/py36/lib/python3.6/site-packages/sh.py", line 2459, in background_thread
    handle_exit_code(exit_code)
  File "/home/gjb/miniconda3/envs/py36/lib/python3.6/site-packages/sh.py", line 2157, in fn
    return self.command.handle_command_exit_code(exit_code)
  File "/home/gjb/miniconda3/envs/py36/lib/python3.6/site-packages/sh.py", line 815, in handle_command_exit_code
    raise exc
sh.SignalException_SIGKILL: 

  RAN: /bin/sleep 60

  STDOUT:


  STDERR:




## Users

You can retrieve information on the users on the system as well.

In [18]:
for user in psutil.users():
    started = datetime.strftime(datetime.fromtimestamp(user.started), '%Y-%m-%d %H:%M:%S')
    print(f'{user.name}: {started}')

gjb: 2018-11-14 20:52:32


## Performance

The `psutil` module makes quite some interesting statistics related to system performance available. This can be useful when writing monitoring tools.

The cumulative times for user, nice, system and so on are readily available.

In [32]:
psutil.cpu_times()

scputimes(user=2992.29, nice=67.76, system=919.06, idle=58311.66, iowait=76.19, irq=0.0, softirq=12.07, steal=0.0, guest=0.0, guest_nice=0.0)

Memory usage can be queried.

In [33]:
psutil.virtual_memory()

svmem(total=4112015360, available=1358479360, percent=67.0, used=2459013120, free=474349568, active=1601912832, inactive=935960576, buffers=344289280, cached=834363392, shared=19922944, slab=262770688)

Disk I/O measures such as the total IOP and read/write sizes in byte are easy to obain.

In [35]:
psutil.disk_io_counters()

sdiskio(read_count=462764, write_count=114006, read_bytes=5862454272, write_bytes=3719344128, read_time=328092, write_time=831264, read_merged_count=72280, write_merged_count=176461, busy_time=213560)

Network I/O can similarly be monitored.

In [49]:
psutil.net_io_counters()

snetio(bytes_sent=21116518, bytes_recv=86326034, packets_sent=61886, packets_recv=102453, errin=0, errout=0, dropin=0, dropout=0)

Disk usage for all partitions can be queried.

In [48]:
for partition in psutil.disk_partitions():
    mountpoint = partition.mountpoint
    if 'snap' not in mountpoint:
        print(f'{mountpoint}: {psutil.disk_usage(mountpoint).percent:.1f}')

/: 77.5
/var/lib/docker/aufs: 77.5
